## ДЗ векторные представления



* Разбейте собранные данные на train/test, отложив 20-30% наблюдений для тестирования.
* Примените tf-idf преобразование для текстового описания. Используйте как
отдельные токены, так и биграммы, отсейте стоп-слова, а также слова, которые встречаются слишком редко или слишком часто (параметры min/max_df), не забудьте убрать l2 регуляризацию, которая по умолчанию включена.
* Обучите random forest или градиентный бустинг (LightGBM или catboost) на полученных векторах и подберите оптимальную комбинацию гиперпараметров с помощью GridSearch
* Теперь воспользуйтесь предобученными word2vec/fasttext эмбеддингами для векторизации текста. Векторизуйте тексты с помощью метода word2vec/fasttext c весами tf-idf (Совет: для текстов на русском языке можно взять предобученные эмбеддинги с сайта rusvectores https://rusvectores.org/ru/models/ (вам подходят эмбеддинги с параметром тэгсет НЕТ). Для английского языка можете воспользоваться word2vec, обученными на Google News)
* Повторите эксперимент из пункта 4 с использованием полученных в пункте 5 векторов

In [ ]:
!pip install pyaspeller
!pip install catboost
!pip install lightgbm
!pip install ufal.udpipe
!pip install wget
!pip install natasha
!pip install pymorphy2
!pip install fasttext
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 936.8/936.8 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=27cf71e8e76caccd7ba0bea8f6058d759c4e63b0aafb9ba4d1a4107fe2f66e9e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=81647200fca163c7d83ce73607f80b4

In [ ]:
import random
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import time as t
import re
import tqdm
from tqdm import notebook
from pyaspeller import YandexSpeller

# nltk

import nltk
from nltk.corpus import stopwords # списки стоп-слов
nltk.download('stopwords') # загрузил список стоп-слов
stop_words = nltk.corpus.stopwords.words('russian')
from sklearn.feature_extraction.text import TfidfVectorizer # импорт TF-IDF
count_tf_idf = TfidfVectorizer(stop_words=list(stop_words)) # счетчик TF-IDF со стоп-словами


# gensim

import gensim
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
from gensim.corpora import Dictionary # создания словаря частотности токенов в тексте
from gensim.models import Phrases # n-граммы
from gensim.models import Word2Vec, TfidfModel, FastText, KeyedVectors


# collections
from collections import defaultdict, Counter
from pymystem3 import Mystem
m = Mystem()


# models

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV # поиск оптимальных параметров
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from category_encoders.cat_boost import CatBoostEncoder

from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    Doc,
    NewsNERTagger,
    MorphVocab,
    PER
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
# загрузка текстов
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving mirf.csv to mirf.csv
User uploaded file "mirf.csv" with length 7321210 bytes


In [ ]:
df = pd.read_csv('mirf.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 39 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   Unnamed: 0                                                       701 non-null    int64 
 1   url                                                              701 non-null    object
 2   author                                                           701 non-null    object
 3   publ_date                                                        701 non-null    object
 4   title                                                            701 non-null    object
 5   description                                                      701 non-null    object
 6   text                                                             701 non-null    object
 7   views                                                

Буду предсказывать оценки книги рецензентом (score). В EDA выяснил, что на оценки влияет жанровая специфика рецензирумой книги, рецензент. Весьма вероятно, что книгам известных фантастов ставят более высокие оценки. Поэтому в векторы текстов рцензии добавим эти переменные (также в виде вектора).

### Дамми коды для жанров книги в вектор

In [ ]:
# список имен столбцов с дамми для жанров
col_gen = df.columns.to_list()[18:38]
col_gen

['АИ-фэнтези',
 'авантюрное фэнтези',
 'боевик',
 'городское фэнтези',
 'детектив',
 'историческое фэнтези / альтернативная-история / хронофантастика',
 'космическая опера',
 'мистика',
 'мифологическое фэнтези',
 'научная / "твердая" фантастика / технофэнтези',
 'ориентальное фэнтези',
 'подростковое фэнтези',
 'постапокалипсис / катастрофа',
 'псевдодокументалистика',
 'путеводитель по вымышленной вселенной',
 'социальная / "мягкая" фантастика / антиутопия / постмодерн',
 'триллер',
 'тёмное / готическое фэнтези / нуар',
 'хоррор',
 'эпическое / приключенческое / героическое фэнтези']

In [ ]:
# значения дамми столбцов с жанрами в матрицу
# затем присоединю к df ее строки
matrix = df[col_gen].to_numpy()

In [ ]:
matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# матрицу в датафрейм
m_df = pd.DataFrame(data=pd.Series(list(matrix)))

In [ ]:
# объединил датафреймы
# назначил матрице индексы от исходного df и join
m_df.index = df.index
df = df.join(m_df)


In [ ]:
df[0][0]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
# переименую получившийся столбец
df = df.rename(columns = {0:'gen_vec'})

### Закодирую столбецы с авторами обзоров и фантастами, на чьи книги эти обзоры
Фантасты, а также название книг, на которую рецензия, - в столбце title

In [ ]:
# функция, которая выделит фантаста, на книгу которого отзыв
#  и запишет его ФИ в отдельный столбец

def fantast_extractor(row):

    text = row['title']
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    for span in doc.spans:
        span.normalize(morph_vocab)

    per_l = [s.normal for s in doc.spans if s.type == 'PER']

    if per_l:
        row['fantast'] = per_l[0]
    else:
        row['fantast'] = ' '

    return row

In [ ]:
# применим функцию выше
df = df.apply(fantast_extractor, axis=1)

In [ ]:
df['fantast'][500]

'Ник Перумов'

In [ ]:
# список категориальных переменных для перекодирования
cat_features = ['author', 'fantast']

In [ ]:
# инициализировал кодировщик
encoder = CatBoostEncoder(cols=cat_features, # какие категориальные фичи перекодировать
                          handle_unknown='value', # как поступать с зн, которых нет в train
                          handle_missing='value', # как с пропускам в категориальных пер
                          random_state=123,
                          sigma=0.01)

#### разделил на train и test (80:20)

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=123)

In [ ]:
# перекодировал

train[cat_features] = encoder.fit_transform(train[cat_features], train['score'])
test[cat_features] = encoder.transform(test[cat_features])

## Текст в вектор

Возьму эмбеддинги из предобученной модели w2v RusVectōrēs. <br>
Fasttext модели RusVectōrēs скачать не вышло. <br>
Качество w2v модели RusVectōrēs для gensim лучше, чем fasttext.load_model('cc.ru.300.bin'), судя по синонимам.


In [ ]:
# загрузка предобученных векторов w2v
# ruscorpora_upos_skipgram_300_10_2017

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving model.bin to model.bin
User uploaded file "model.bin" with length 231349895 bytes


In [ ]:
model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [ ]:
# хорошая функция для спелчекинга (для разбивки слипшихся при парсинге слов)
# YandexSpeller хорошо разбивает слипшиеся слова, если давать ему текст кусками,
# по 5-7 слов, иначе он их пропускает
# но она очень долго работает

def speller_castom(text):

    tokens = re.sub(r'[^А-Яа-яA-zёЁ.!?-]', ' ', text).split()
    text = " ".join(tokens)
    cl_chunks = [YandexSpeller().spelled(ch) for ch in textwrap.wrap(text, 55)]

    return " ".join(cl_chunks)

In [ ]:
# POS таги для лемм не из числа стоп-слов и длинна которых > 3 + очистка
# здесь же спеллер на текст целиком и удаление лишних символов

def token_POS (text):
    text = text.lower()
    tokens = re.sub(r'[^А-Яа-яA-zёЁ.!?-]', ' ', text).split()
    text = " ".join(tokens)
    text = YandexSpeller().spelled(text)


    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    for span in doc.spans:
        span.normalize(morph_vocab)

    return ['{}_{}'.format(t.lemma, t.pos) for t in doc.tokens if (t not in stop_words and len(t.lemma) > 3)]

In [ ]:
# применил
train['cl_tk_pos'] = train['text'].apply(token_POS)
test['cl_tk_pos'] = test['text'].apply(token_POS)

In [ ]:
train.head(10)

,Unnamed: 0,url,author,publ_date,title,description,text,views,year,month_year,...,путеводитель по вымышленной вселенной,"социальная / ""мягкая"" фантастика / антиутопия / постмодерн",триллер,тёмное / готическое фэнтези / нуар,хоррор,эпическое / приключенческое / героическое фэнтези,юмористическое фэнтези,gen_vec,fantast,cl_tk_pos
344,344,https://www.mirf.ru/book/cezary-zbierzchowski-...,7.588145,2019-09-18,Цезарий Збежховский «Всесожжение»,"Апокалипсис наступил: Бог покинул Землю, кора...",Наступление польской фантастики нанаши полки ...,915,2019,2019-09,...,0,1,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",7.663245,"[наступление_NOUN, польский_ADJ, фантастика_NO..."
252,252,https://www.mirf.ru/book/sbornik-nochnye-viden...,7.747939,2020-08-17,Сборник «Ночные видения»: отличный хоррор про ...,"Если сказать, что у любого автора хоррора ест...",Большая часть сборника посвящена традициям иа...,804,2020,2020-08,...,0,0,0,0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.688786,"[больший_ADJ, часть_NOUN, сборник_NOUN, посвят..."
332,332,https://www.mirf.ru/book/robert-dzhekson-benne...,8.359303,2019-10-29,Роберт Джексон Беннетт «Город чудес»,После событий в Вуртьястане прошло тринадцать...,"Напервый взгляд, «Город чудес» выстроен также...",1283,2019,2019-10,...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",7.558430,"[напервый_ADJ, взгляд_NOUN, город_NOUN, чудо_N..."
564,564,https://www.mirf.ru/book/michael-swanwick-poly...,7.555874,2016-12-05,Майкл Суэнвик «Полёт феникса»,"В мире, распавшемся на куски после глобальной ...",Учёные-китаисты оторвут мне голову за такое у...,3202,2016,2016-12,...,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",7.671891,"[ученый-китаист_ADJ, оторвать_VERB, голова_NOU..."
491,491,https://www.mirf.ru/book/diana-uinn-dzhons-str...,8.176227,2017-09-27,Диана Уинн Джонс «Страшная тайна»,Молодой магид Руперт Венейблз должен подыскат...,Творческое наследие покойной Дианы Уинн Джонс...,3065,2017,2017-09,...,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.803073,"[творческий_ADJ, наследие_NOUN, покойный_ADJ, ..."
250,250,https://www.mirf.ru/book/filip-pulman-tajnoe-s...,7.798117,2020-08-30,Филип Пулман «Тайное содружество»: герои «Тёмн...,Лира Белаква-Сирин уже не маленькая девочка: ...,Смотрите также Филип Пулман «Тайное содружест...,896,2020,2020-08,...,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.771786,"[смотрите_NOUN, также_ADV, филип_NOUN, пулман_..."
120,120,https://www.mirf.ru/book/brom-kosinog-istoriya...,7.485268,2022-02-18,Бром «Косиног. История о колдовстве». Милый Дь...,1666 год. Отец продаёт юную Абиту в жёны благ...,"Образ Абиты, пожалуй, главная удача романа. Ж...",2729,2022,2022-02,...,0,0,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",8.253606,"[образ_NOUN, абит_NOUN, пожалуй_ADV, главный_A..."
150,150,https://www.mirf.ru/book/alaster-rejnolds-alma...,7.638525,2021-10-13,Аластер Рейнольдс «Алмазные псы». Летопись «Пр...,"Эксцентричный исследователь, считавшийся давн...",Magnum opus Рейнольдса ныне включает в себя с...,978,2021,2021-10,...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",7.710222,"[magnum_X, opus_X, рейнольдс_NOUN, ныне_ADV, в..."
458,458,https://www.mirf.ru/book/benedikt-dzheka-aleks...,8.441239,2018-03-22,Бенедикт Джэка «Алекс Верус. Участь»,"Тридцатилетний прорицатель Алекс Верус, бывши...","Роман «Участь» начинается с удачной фразы, ко...",1850,2018,2018-03,...,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.690452,"[роман_NOUN, участь_NOUN, начинаться_VERB, уда..."
626,626,https://www.mirf.ru/book/bookreview/nick-perum...,7.604937,2016-03-10,Ник Перумов «Приключения Молли Блэкуотер. За к...,"Двенадцатилетняя Молли Блэкуотер, благовоспита...","Оставив в стороне привычные темы, Ник Перумов...",9535,2016,2016-03,...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",7.734119,"[оставить_VERB, сторона_NOUN,

Векторы w2v слов в тексте сложу и нормирую на idf, чтобы получить векторы текстов. Для этого обучу tf-idf на общем корпусе из train и test, чтобы не встречались токены, которых нет в словаре модели.

In [ ]:
# создам общий корпус в котором леммы с POS
train_cor = train['cl_tk_pos'].sum()
test_cor = test['cl_tk_pos'].sum()
corpus = train_cor + test_cor

In [ ]:
# счётчик, указав в нём стоп-слова:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf = count_tf_idf.fit(corpus)
# словарь {ключ(токен):значение(idf)}
tf_idf_dict = dict(zip(tf_idf.get_feature_names_out(), np.float16(tf_idf.idf_)))

In [ ]:
# функция, которая вернет средний вектор для текстов w2v нормированный на idf

def mean_text_idf_vector(text):
  return np.mean( np.array([ model.get_vector(t) * np.float16(tf_idf_dict.get(t.lower())) for t in text\
                            if (t in model.key_to_index.keys() and '-' not in t) ]), axis=0 )

In [ ]:
# применил функцию выше
train['txt_vec'] = train['cl_tk_pos'].apply(mean_text_idf_vector)

In [ ]:
train.sample(10)

,Unnamed: 0,url,author,publ_date,title,description,text,views,year,month_year,...,"социальная / ""мягкая"" фантастика / антиутопия / постмодерн",триллер,тёмное / готическое фэнтези / нуар,хоррор,эпическое / приключенческое / героическое фэнтези,юмористическое фэнтези,gen_vec,fantast,cl_tk_pos,txt_vec
583,583,https://www.mirf.ru/book/ken-lyu-korolevskie-m...,7.869562,2016-09-25,Кен Лю «Королевские милости»,"Во главе восстания против императора Мапидэрэ,...",«Королевские милости» — дебютный роман Кена Л...,5447,2016,2016-09,...,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",7.711935,"[королевский_ADJ, милость_NOUN, дебютный_ADJ, ...","[5.035576820373535, 0.6799318194389343, 3.9131..."
566,566,https://www.mirf.ru/book/evgenij-lukin-grekhi-...,8.306202,2016-11-27,Евгений Лукин «Грехи наши тяжкие»,Как строит свои сюжеты классический научный фа...,"Волгоградский писатель Евгений Лукин, лауреат...",3200,2016,2016-11,...,1,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.756295,"[волгоградский_ADJ, писатель_NOUN, евгений_ADJ...","[3.745849370956421, 0.6793091893196106, 4.0566..."
413,413,https://www.mirf.ru/book/fonda-lee-nefritovyj-...,7.646271,2018-10-30,Фонда Ли «Нефритовый город»,Островное государство Кекон — единственное ме...,"Впоследние годы немало авторов, сделавших себ...",5701,2018,2018-10,...,0,0,0,0,0,0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.411628,"[впоследний_ADJ, немало_ADV, автор_NOUN, сдела...","[1.3047658205032349, 1.3431161642074585, 5.574..."
354,354,https://www.mirf.ru/book/klajv-barker-alye-pes...,7.478916,2019-07-30,Клайв Баркер «Алые песнопения»,"Гарри Д’Амура, оккультного детектива из Нью-Й...",«Алые песнопения» продолжение повести «Восста...,1436,2019,2019-07,...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.724081,"[алый_ADJ, песнопение_NOUN, продолжение_NOUN, ...","[-0.5688296556472778, -0.7515737414360046, 1.9..."
120,120,https://www.mirf.ru/book/brom-kosinog-istoriya...,7.485268,2022-02-18,Бром «Косиног. История о колдовстве». Милый Дь...,1666 год. Отец продаёт юную Абиту в жёны благ...,"Образ Абиты, пожалуй, главная удача романа. Ж...",2729,2022,2022-02,...,0,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",8.253606,"[образ_NOUN, абит_NOUN, пожалуй_ADV, главный_A...","[0.6142469644546509, -0.9924155473709106, 2.18..."
663,663,https://www.mirf.ru/book/kikuchi-hideyuki-sobr...,8.330767,2015-05-07,Хидеюки Кикути «Собрание призрачного меча»,Жизнь самурая подчинена строгим и безжалостным...,"Когда мы беседуем о зарубежной фантастике, ра...",3266,2015,2015-05,...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, ...",7.709416,"[когда_SCONJ, беседовать_VERB, зарубежный_ADJ,...","[0.5269727110862732, -0.2753143012523651, 1.00..."
361,361,https://www.mirf.ru/book/sergej-lukyanenko-porog/,7.545748,2019-06-13,Сергей Лукьяненко «Порог»: фантастика в духе «...,"XXIII столетие. В экспедицию к системе Невар,...",Спустя довольно длительное время Сергей Лукья...,29238,2019,2019-06,...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...",7.133045,"[спустя_ADP, довольно_ADV, длительный_ADJ, вре...","[0.24082523584365845, -3.1358609199523926, 6.5..."
256,256,https://www.mirf.ru/book/glen-kuk-portal-tenej/,6.714870,2020-08-09,Глен Кук «Портал теней»: возвращение Черного О...,Солдаты Чёрного Отряда не задают вопросов. Он...,"Глен Кук впервые упомянул, что «Чёрный Отряд»...",1705,2020,2020-08,...,0,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",7.630920,"[глен_NOUN, впервые_ADV, упомянуть_VERB, черны...","[0.2863427996635437, 2.9102394580841064, 3.263..."
136,136,https://www.mirf.ru/book/robert-makkammon-gran...,6.520213,2021-12-02,Роберт Маккаммон «Граница». Поле битвы — Земля!,Беда пришла откуда не ждали: человеческую цив...,"На первый взгляд кажется, что «Границе» сложн...",1154,2021,2021-12,...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ...",7.855091,"[первый_ADJ, взгляд_NOUN, казаться_VERB, гр

In [ ]:
test['txt_vec'] = test['cl_tk_pos'].apply(mean_text_idf_vector)

In [ ]:
# добавил вектор рецензента, фантаста, жанра в вектор текста
# длинна 1 + 1 + 20 + 300 = 322

# for i in train.index:
#   train['txt_vec'][i] = np.append(train['txt_vec'][i], [train['author'][i], train['fantast'][i], train[0][i]])

for i in train.index:
  train['txt_vec'][i] = np.append(train['txt_vec'][i], train['author'][i])
  train['txt_vec'][i] = np.append(train['txt_vec'][i], train['fantast'][i])
  train['txt_vec'][i] = np.append(train['txt_vec'][i], train['gen_vec'][i])

In [ ]:
len(train['txt_vec'][500])

322

In [ ]:
# то же для test

for i in test.index:
  test['txt_vec'][i] = np.append(test['txt_vec'][i], test['author'][i])
  test['txt_vec'][i] = np.append(test['txt_vec'][i], test['fantast'][i])
  test['txt_vec'][i] = np.append(test['txt_vec'][i], test['gen_vec'][i])

In [ ]:
train['score'][501]

8

## Прогноз оценки в отзве по дополненному вектору текста

Оценки от 0 до 10 (после изменений в EDA от 5 до 10). Это задача регрессии. <br> Оценка всегда целое чсило, поэтому она может быть решена и как задача классфикации (5 классов). <br><br>
### (!) Грабли: сохранять как csv df c np.array в столбце нельзя, т.к. при загрузке пропадут все запятые, разделяющие элементы массива.

In [ ]:
X_train = train['txt_vec'].tolist()
y_train = train['score'].tolist()

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_test = test['txt_vec'].tolist()
y_test = test['score'].tolist()

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_train


array([[ 3.88802242,  0.73818123,  4.42710161, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.83842683,  0.27181032,  1.3118633 , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.30965161,  1.31024992,  0.27054003, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.42284465,  0.18144356,  3.45218968, ...,  0.        ,
         0.        ,  0.        ],
       [ 2.2948339 , -1.71630466,  2.12299347, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.46467102, -0.95052743,  3.29318833, ...,  1.        ,
         0.        ,  1.        ]])

In [ ]:
# задал модель
# в качестве функции потерь взял RMSE
embedding_features=[0]


model = CatBoostRegressor(loss_function="RMSE")
parametrs = { 'learning_rate': [0.01, 0.2, 0.5],
              'iterations': range (10, 51, 10),
              'depth': range (2, 10, 2) }

In [ ]:
# подбор гиперпараметров
r_search = RandomizedSearchCV(model,
                              parametrs,
                              cv=3,
                              scoring='neg_root_mean_squared_error',
                              random_state=123,
                              n_iter=10,
                              verbose=1,
                              n_jobs=-1)

In [ ]:
r_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
0:	learn: 1.1876862	total: 13.9ms	remaining: 682ms
1:	learn: 1.1819906	total: 25.6ms	remaining: 613ms
2:	learn: 1.1678674	total: 36.5ms	remaining: 572ms
3:	learn: 1.1519404	total: 47.4ms	remaining: 545ms
4:	learn: 1.1478270	total: 58ms	remaining: 522ms
5:	learn: 1.1405617	total: 68.6ms	remaining: 503ms
6:	learn: 1.1339388	total: 78.8ms	remaining: 484ms
7:	learn: 1.1249517	total: 88.8ms	remaining: 466ms
8:	learn: 1.1178029	total: 98.8ms	remaining: 450ms
9:	learn: 1.1064283	total: 109ms	remaining: 434ms
10:	learn: 1.1007425	total: 119ms	remaining: 423ms
11:	learn: 1.0913791	total: 129ms	remaining: 409ms
12:	learn: 1.0900735	total: 139ms	remaining: 396ms
13:	learn: 1.0828391	total: 149ms	remaining: 383ms
14:	learn: 1.0744775	total: 159ms	remaining: 370ms
15:	learn: 1.0692290	total: 169ms	remaining: 359ms
16:	learn: 1.0669978	total: 179ms	remaining: 347ms
17:	learn: 1.0605440	total: 189ms	remaining: 335ms
18:	learn: 1.0583735	tot

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7be314fc0c10>,
                   n_jobs=-1,
                   param_distributions={'depth': range(2, 10, 2),
                                        'iterations': range(10, 51, 10),
                                        'learning_rate': [0.01, 0.2, 0.5]},
                   random_state=123, scoring='neg_root_mean_squared_error',
                   verbose=1)

In [ ]:
# лучшие параметры модели
r_search.best_params_

{'learning_rate': 0.2, 'iterations': 50, 'depth': 2}

In [ ]:
#cv_results_ в таблицу
results = pd.DataFrame(r_search.cv_results_)
# сохранил индекс лучшей модели
best_index = r_search.best_index_
# сохранил RMSE лучшей модели по модулю, т.к. scoring='neg_root_mean_squared_error'
RMSE = round(abs(results['mean_test_score'].iloc[0]))

print(f"RMSE: {RMSE}")

RMSE: 1


In [ ]:
CatBoost = CatBoostRegressor(loss_function="RMSE", **r_search.best_params_)

In [ ]:
CatBoost.fit(X_train, y_train)

0:	learn: 1.2012342	total: 61.2ms	remaining: 2.38s
1:	learn: 1.1998978	total: 115ms	remaining: 2.19s
2:	learn: 1.1988791	total: 170ms	remaining: 2.09s
3:	learn: 1.1978935	total: 229ms	remaining: 2.06s
4:	learn: 1.1965038	total: 284ms	remaining: 1.99s
5:	learn: 1.1957988	total: 328ms	remaining: 1.86s
6:	learn: 1.1946581	total: 363ms	remaining: 1.71s
7:	learn: 1.1935073	total: 400ms	remaining: 1.6s
8:	learn: 1.1924313	total: 455ms	remaining: 1.57s
9:	learn: 1.1918225	total: 512ms	remaining: 1.53s
10:	learn: 1.1904404	total: 573ms	remaining: 1.51s
11:	learn: 1.1894704	total: 628ms	remaining: 1.47s
12:	learn: 1.1886716	total: 680ms	remaining: 1.41s
13:	learn: 1.1875381	total: 743ms	remaining: 1.38s
14:	learn: 1.1864723	total: 801ms	remaining: 1.33s
15:	learn: 1.1855809	total: 864ms	remaining: 1.3s
16:	learn: 1.1850000	total: 919ms	remaining: 1.24s
17:	learn: 1.1842913	total: 979ms	remaining: 1.2s
18:	learn: 1.1828136	total: 1.02s	remaining: 1.13s
19:	learn: 1.1819094	total: 1.06s	remaining

In [ ]:
print('RMSE модели на тестовой выборке',
      round(mean_squared_error(y_test, CatBoost.predict(X_test))**0.5))
print('RMSE модели на обучающей выборке',
      round(mean_squared_error(y_train, CatBoost.predict(X_train))**0.5))

RMSE модели на тестовой выборке 1
RMSE модели на обучающей выборке 1
